# Flattened 1-D Multi-Input Channel Convolution

This notebook uses the fiber-tree emulator to display the behaviour of flattening multiple input channel 1-D into a simple 1-D convolution of a single channel. 

Note this notebook relies on the invariant that the data is dense. Therefore, because the data is assumed to be dense we can use the position-based operators on the premise that for dense data the position and coordinate are the same.

First, include some libraries

In [ ]:
# Begin - startup boilerplate code

import pkgutil

if 'fibertree_bootstrap' not in [pkg.name for pkg in pkgutil.iter_modules()]:
  !python3 -m pip  install git+https://github.com/Fibertree-project/fibertree-bootstrap --quiet

# End - startup boilerplate code

from fibertree_bootstrap import *

fibertree_bootstrap(style="uncompressed", animation="movie")

## Logging Control

In [ ]:
enable_log = False

def set_params(log):
    global enable_log

    enable_log = (log == 'enable')

def log(*args):
    if enable_log:
        print(*args)

controls = interactive(set_params,
                       log=['disable', 'enable'])


display(controls)

## Convolution Inputs Selection

Using sliders to select the shapes of the weights and input activations

In [ ]:
C = 2
S = 3
W = 8


tm = TensorMaker("toeplitz")

tm.addTensor("I_CW",
             rank_ids=["C", "W"],
             shape=[C, W],
             density=1,
             interval=5,
             seed=0,
             color="blue")

tm.addTensor("F_CS",
             rank_ids=["C", "S"],
             shape=[C, S],
             density=1,
             interval=5,
             seed=0,
             color="green")

tm.displayControls()

## Create Input Tensors

Given shapes selected above create and display the filter weights (**f**) and input activations (**i**) and a reference output (**o_verify**)

In [ ]:
I_CW = tm.makeTensor("I_CW")
F_CS = tm.makeTensor("F_CS")

print("Input activations")
displayTensor(I_CW)
print("Filter weights")
displayTensor(F_CS)

C = I_CW.getShape("C")
W = I_CW.getShape("W")
S = F_CS.getShape("S")

Q = W - S + 1

print("")
print(f"Output activations - {Q}")

## Verification Convolution

In [ ]:
i_raw = I_CW.getRoot().uncompress()
f_raw = F_CS.getRoot().uncompress()
o_verify_raw = Q*[0]

print(f"I: {i_raw}")
print(f"F: {f_raw}")

for q in range(Q):
    for c in range(C):
        for s in range(S):
            w = q + s
            o_verify_raw[q] += i_raw[c][w] * f_raw[c][s]
        
print(f"O_verify: {o_verify_raw}")

o_verify = Tensor.fromUncompressed(name="O_verify",
                                   rank_ids=["Q"],
                                   shape=[Q],
                                   root=o_verify_raw,
                                   default=None)

displayTensor(o_verify)

## Multiple Input Channel Convolution

Process the convolution of the multi-channel input activation and filter weight tensors.

In [ ]:
o = Tensor.makePopulated(name="O",
                         rank_ids=["Q"],
                         shape=[Q])

print("Convolution")

i = I_CW
f = F_CS

canvas = createCanvas(f, i, o)

for q in range(Q):
    log(f"Processing output: ({q}, ({o[q]}))")
    for c in range(C):
        for w in range(W):
            s = w - q
            if s < 0 or s >= S: continue
            log(f"Processing input: ({c}, {w}, {i[c][w]})")
            log(f"  Processing filter weight ({c}, {s}, {f[c][s]})")
            o[q] += f[c][s] * i[c][w]

            canvas.addActivity((), [(c, w,) for w in range(q, q+S)], (), worker="W")
            canvas.addFrame((c, s,), (c, w,), (q,))

displayTensor(o)
displayCanvas(canvas)

print("Input Activations - before")
assert o == o_verify

## Flatten Inputs and Filters

Flatten the input activations and filter weights into a single input channel.

Note: The rank_id and shape of the tensor are changed as the coordinates are updated.

In [ ]:
print("Filter Weights - before")
displayTensor(f)

f_flat = f.swapRanks().flattenRanks()
f_flat.getRoot().updateCoords(lambda pos, c, p: (c[0]*C)+c[1],
                              new_rank_id="CS",
                              new_shape=C*S)
f_flat.setName("F_flat")

print("Filter Weights - after")
displayTensor(f_flat)


print("Input Activations - before")
displayTensor(i)

i_flat = i.swapRanks().flattenRanks()
i_flat.getRoot().updateCoords(lambda pos, c, p: (c[0]*C)+c[1],
                              new_rank_id="CW",
                              new_shape=C*W)
i_flat.setName("I_flat")

print("Input Activations - after")
displayTensor(i_flat)




## Flattened convolution

Process a convolution on the flattened input activations and filter weights. Note that the window now slides by ```C``` the number of inputs channels.

Note, this processing pattern is the same as used by Eyeriss for processing multiple input channels at once in a single PE.

In [ ]:
o = Tensor.makePopulated(name="O",
                         rank_ids=["Q"],
                         shape=[Q])


print("Convolution")


canvas = createCanvas(f_flat, i_flat, o)

for q in range(Q):
    log(f"Processing output: ({q}, ({o[q]}))")
    for cw in range(C*W):
        cs = cw - C*q
        if cs < 0 or cs >= C*S: continue
        log(f"Processing input: ({cw}, {i_flat[cw]})")
        log(f"  Processing filter weight ({cs}, {f_flat[cs]})")
        o[q] += f_flat[cs] * i_flat[cw]

        i_window = [(cw,) for cw in range(C*q, C*q+C*S)]
        canvas.addActivity((), i_window, (), spacetime=("W", (q, cw)))
        canvas.addActivity((cs,), (cw,), (q,), spacetime=("P", (q, cw)))

displayTensor(o)
displayCanvas(canvas)

assert o == o_verify

## Testing area

For running alternative algorithms